# Folsom

Simple test case simulates folsom dam operations from 1 Oct 2000 - 30 Sept 2010 using simple rule curve, maximum flood release. Conservation pool demands are from outflow data when the reservoir storage is below top of conservation. Evaporation data is used to better fit observations.

In [1]:
from pathlib import Path
import pandas as pd

In [2]:
data_dir = Path(r"c:\Users\kucharsk\OneDrive - Stichting Deltares\Documents\projects\ribasim\folsom")

In [3]:
inflows = pd.read_excel(data_dir / "inflows_daily-2000-2010.xlsx", engine="openpyxl",
                        index_col="DATE TIME", parse_dates=True, date_format="%Y-%m-%d %H:%M",
                        usecols=["DATE TIME", "VALUE"], thousands=",").rename(columns={"VALUE": "inflow_cfs"})
storage = pd.read_excel(data_dir / "storage_daily-2000-2010.xlsx", engine="openpyxl",
                        index_col="DATE TIME", parse_dates=True, date_format="%Y-%m-%d %H:%M",
                        usecols=["DATE TIME", "VALUE"], thousands=",").rename(columns={"VALUE": "storage_af"})
outflows = pd.read_excel(data_dir / "outflow_daily-2000-2010.xlsx", engine="openpyxl",
                        index_col="DATE TIME", parse_dates=True, date_format="%Y-%m-%d %H:%M",
                        usecols=["DATE TIME", "VALUE"], thousands=",").rename(columns={"VALUE": "outflow_cfs"})
evap = pd.read_excel(data_dir / "evap_daily-2000-2010.xlsx", engine="openpyxl",
                     index_col="DATE TIME", parse_dates=True, date_format="%Y-%m-%d %H:%M",
                     usecols=["DATE TIME", "VALUE"], thousands=",").rename(columns={"VALUE": "evap_cfs"})
df = inflows.join([storage, outflows, evap])
df.head()

,inflow_cfs,storage_af,outflow_cfs,evap_cfs
DATE TIME,,,,
2000-10-01,1107.0,659258,1849.0,9
2000-10-02,1432.0,658149,1991.0,0
2000-10-03,1357.0,656208,2327.0,9
2000-10-04,631.0,653436,2020.0,9
2000-10-05,908.0,651034,2110.0,9


In [4]:
df_nan = df[df.isna().any(axis=1)]
df_nan.head(10)

,inflow_cfs,storage_af,outflow_cfs,evap_cfs
DATE TIME,,,,
2004-05-07,NaN,640600,NaN,64
2004-05-08,NaN,639800,NaN,58
2005-08-13,NaN,800600,NaN,115
2006-03-30,NaN,704800,NaN,32
2007-10-11,NaN,309500,NaN,15
2008-01-04,NaN,225800,NaN,0
2008-12-05,NaN,202200,NaN,4
2009-12-15,NaN,281100,NaN,2


These observations are missing inflow and outflow values. Therefore, they cannot be filled in with: S_t = S_t-1 + I_t - R_t. 

So, the missing inflow values are interpolated from the previous good value and subsequent good value. 

Missing outflows (R_t) are computed as: R_t = S_t-1 - S_t + I_t.

In [6]:
df_neg = df[(df < 0).any(axis=1)]
df_neg.head(10)

,inflow_cfs,storage_af,outflow_cfs,evap_cfs
DATE TIME,,,,
2000-10-22,-112.0,594705,2276.0,9


This value can be fixed by using: S_t = S_t-1 + I_t - R_t.

Rearranged: I_t = S_t - S_t-1 + R_t

In [ ]:
def is_conserving_mass(i: int, row: pd.Series):
    previous_storage = df.iloc[i - 1]["storage_af"]
    storage = previous_storage + row["inflow_cfs"] - row["outflow_cfs"] - row["evap_cfs"]
    return row["inflow_cfs"] - row["outflow_cfs"] - row["evap_cfs"] == row["storage_af"]

In [ ]:
interpolate: bool = False
last, next, n = 0, 0, 0
for i, row in df.iterrows():
    if row["inflow_cfs"].isna():
        if not interpolate:
            n += 1
            last = df.iloc[i - 1]["inflow_cfs"]
    else:
        if interpolate: